In [ ]:
# Import packages
import tensorflow as tf
import os
import cv2
import imghdr
import numpy as np
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
# Limiting the GPU memory consumption growth (to avoid out of memory errors)
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
# Extract information about the directory so that the images can be read
data_dir = "CNN Images"
os.remove(os.path.join(data_dir, '.DS_Store'))
image_exts = ['jpeg', 'jpg', 'bmp', 'png']

In [ ]:
# Cycle through each image in each directory and check to see if they can be opened and if they match the accepted fire extnesions (image_exts)
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                os.remove(image_path)
        except:
            os.remove(image_path)

In [ ]:
# Define data using keras- the defult size will be 256x256 with 32 images within each batch
# Keras will also autoshuffle the dataset
data = tf.keras.utils.image_dataset_from_directory('Images')

In [ ]:
# create a data iterator so that we can move through the batches created by keras
data_iterator = data.as_numpy_iterator()

In [ ]:
# cycle through the batchs
batch = data_iterator.next()

In [ ]:
# Batch information
batch[0].shape

In [ ]:
# Visualise the first 4 images of the batch showing the class numebr
fig, ax = plt.subplots(ncols= 4, figsize= (20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

In [ ]:
# Scale the data so that the intensity values (0-225) are scaled between 0 and 1
data_scaled = data.map(lambda x,y: (x/255, y))

In [ ]:
# create a data iterator so that we can move through the batches created by keras
data_iterator_scaled = data_scaled.as_numpy_iterator()

In [ ]:
# cycle through the batchs
batch_scaled = data_iterator_scaled.next()

In [ ]:
batch_scaled[0].shape

In [ ]:
# Visualise the first 4 images of the batch showing the class numebr
fig, ax = plt.subplots(ncols= 4, figsize= (20,20))
for idx, img in enumerate(batch_scaled[0][:4]):
    ax[idx].imshow(img)
    ax[idx].title.set_text(batch_scaled[1][idx])

In [ ]:
# Split the data into training, validation and test datasets
train_size = int(len(data_scaled)*0.7)
val_size = int(len(data_scaled)*0.2)
test_size = int(len(data_scaled)*0.1)

In [ ]:
# Evaluating the sizes- the split may need to be adjusted if len(data_scaled) != train_size+val_size+test_size
print(len(data_scaled))
print(train_size+val_size+test_size)
print(train_size)
print(val_size)
print(test_size)

In [ ]:
# Creating datasets
training_dataset = data_scaled.take(train_size)
val_dataset = data_scaled.skip(train_size).take(val_size)
test_dataset = data_scaled.skip(train_size+val_size).take(test_size)

In [ ]:
# Model container
model = Sequential()

In [ ]:
# Build the CNN
model.add(Conv2D(16,3,1,activation='relu',input_shape=(256,256,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,3,1,activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(16,3,1,activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3,activation='softmax')) #3 classes

In [ ]:
# Compile the model
model.compile('adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# Key metrics are logged within the log directory
logdir='logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
# Train the model
history = model.fit(training_dataset, epochs=20, validation_data=val_dataset, callbacks=tensorboard_callback)

In [ ]:
# Evaluate loss
fig = plt.figure()
plt.plot(history.history['loss'],color='teal',label='loss')
plt.plot(history.history['val_loss'],color='orange',label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
# Evaluate accuracy
fig = plt.figure()
plt.plot(history.history['accuracy'],color='teal',label='accuracy')
plt.plot(history.history['val_accuracy'],color='orange',label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
# Assess the precision, recall and accuracy of the model
precision_metric = Precision()
recall_metric = Recall()
accuracy_metric = BinaryAccuracy()

for batch in test_dataset.as_numpy_iterator():
    x,y = batch
    yhat = model.predict(x)
    yhat= yhat.argmax(axis=1)
    precision_metric.update_state(y, yhat)
    recall_metric.update_state(y, yhat)
    accuracy_metric.update_state(y, yhat)

print(f"Precision: {precision_metric.result().numpy()}, Recall: {recall_metric.result().numpy()}, Accuracy: {accuracy_metric.result().numpy()}")

In [ ]:
# Try the model
def test_image_cnn(img_dir):
    '''Test the CNN
    Expects: String: Image directory
    Modifies: Resizes image to 256x256 and scales pixel vales between 0 and 1
    Returns: String: Animal, Building, or Nature based on probabilities
    '''
    test_image = cv2.cvtColor(cv2.imread(img_dir), cv2.COLOR_BGR2RGB)
    test_image_resize = tf.image.resize(test_image, (256,256))
    plt.imshow(test_image_resize.numpy().astype(int))
    plt.show()

    result = model.predict(np.expand_dims(test_image_resize/255,0))
    indices = np.argsort(result[0])[-3:]

    for i in indices:
        if result[0][i] >= 0.1:
            if i == 0:
                print(f"Animal: {result[0][i]}")
            if i == 1:
                print(f"Building: {result[0][i]}")
            if i == 2:
                print(f"Nature {result[0][i]}")

In [ ]:
# Save the model
model.save('CNN.h5')